In [ ]:
from re import split
!pip install langchain_community langchain_huggingface sentence_transformers pdfplumber

In [ ]:
import pdfplumber
from collections import defaultdict
import itertools
import re
import os
import pandas as pd
from langchain_core.documents import Document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
record_file = 'rag_example.pdf'

In [ ]:
loader = PyPDFLoader(record_file)
documents = loader.load()
filtered_doc = documents[1:3]

In [ ]:
filtered_doc

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
chunking_data = text_splitter.split_documents(filtered_doc)

In [ ]:
chunking_data

In [ ]:
tables = []
with pdfplumber.open(record_file) as pdf:
    for page in pdf.pages:
        table = page.extract_table()
        if table:
            df = pd.DataFrame(table[1:], columns=table[0])
            tables.append(df)

In [ ]:
len(tables)

In [ ]:
tables[0]

In [ ]:
filtered_tables = tables[7:21]
text_dict = defaultdict()
text = ""

In [ ]:
for i, table in enumerate(filtered_tables):
    if isinstance(table, pd.DataFrame):
        table = table.ffill() # 데이터 채우기(이전 행의 값 그대로 사용)
        table = table.apply(lambda x: table.columns + ":" + x.astype(str), axis=1)
        tmp_text = table.to_csv(index=False, header=False)
        tmp_text = re.sub(r'\n"보 장', 'TEMP_REPLACE', tmp_text)
        tmp_text = re.sub(r'\n보 장', 'TEMP_2REPLACE', tmp_text)
        tmp_text = re.sub(r'\n', ' ', tmp_text)
        tmp_text = re.sub(r'TEMP_REPLACE', '\n"보 장', tmp_text)
        tmp_text = re.sub(r'TEMP_2REPLACE', '\n"보 장', tmp_text)
        text_dict[i] = tmp_text.split("\n")
        text += tmp_text

In [ ]:
text_list = list(itertools.chain(*text_dict.values()))

In [ ]:
chunking_data = [Document(page_content=page, metadata=dict(page=i)) for i, page in enumerate(text_list)]